# PS4: A Contextual Stochastic Bandit Personal Shopper
Fill me in

## Consumer choice problem
Imagine that a consumer must choose $m$ possible goods (where each good is in a category with $k$ alternatives) $\mathbf{n} = \left\{n_{1},n_{2},\dots,n_{m}\right\}$, where $n_{j}\in\mathbb{R}_{\geq\epsilon}$ is the quantity of good $j$ chosen where $\epsilon>0$, i.e., the consumer must choose at least $\epsilon$ units of any good. Different combinations of goods are _scored_ using a utility function $U:\mathbb{R}^{m}\rightarrow\mathbb{R}$. In this case, let's assume our consumer uses [the Cobb-Douglas](https://en.wikipedia.org/wiki/Cobb%E2%80%93Douglas_production_function) utility:
$$
\begin{align*}
U(\mathbf{n}) = \prod_{i=1}^{m}n_{i}^{\gamma_{i}}
\end{align*}
$$
where $\gamma_{i} = \left\{-1,1\right\}$ denote _user sentiment_ parameters: if $\gamma_{j} = 1$, the good $j$ is preferred, otherwise is $\gamma_{j} = -1$ good $j$ is _not_ preferred. Finally, the choice of goods $n_{1},\dots,n_{m}$ is subject to a budget constraint:
$$
\begin{align*}
\sum_{i=1}^{m}n_{i}C_{i} \leq B
\end{align*}
$$
where $C_{i}$ is the unit cost of good $i$, and $B$ is the total budget the consumer can spend. The objective of a consumer is to maximize the utility of their choice (the combination of goods) subject to a budget constraint.

## Stochastic Multi-Armed Bandits
In the stochastic multi-armed bandit problem, an agent must choose an action $a$ from the set of all possible actions $\mathcal{A}$, where $\dim\mathcal{A} = K$ during each round $t = 1,2,\dots, T$ of a decision task. The agent chooses action $a\in\mathcal{A}$ and receives a reward $r_{a}$ from the environment, where $r_{a}$ is sampled from some (unknown) distribution $\mathcal{D}_{a}$.

For $t = 1,2,\dots,T$:
1. _Aggregator_: The agent picks an action $a_{t} \in \mathcal{A}$ at time time $t$. How the agent makes this choice is one of the main differences between the different algorithms for solving this problem. 
2. _Adversary_: The agent implements action $a_{t}$ and receives a (random) reward $r_{a}\sim\mathcal{D}_{a}$ where $r_{t}\in\left[0,1\right]$. The distribution $\mathcal{D}_{a}$ is only known to the adversary.
3. The agent updates its _memory_ with the reward and continues to the next decision task. 

The agent is interested in learning the mean of the reward distribution of each arm, $\mu(a) = \mathbb{E}\left[r_{t}\sim\mathcal{D}_{a}\right]$, by experimenting against the world (adversary). 
* __Goal__: The goal of the agent is to maximize the total reward. However, the goal of the algorithm designer is to minimize the _regret_ of the algorithm that the agent uses to choose $a\in\mathcal{A}$.

## Task 1: Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [1]:
include("Include.jl");

First, let's build the `world(...)` function. 
* This function takes the action vector `a::Array{Int64,1}` (the indexes of the goods chosen from each of the $m$ categories), the amount of each good selected from each category from our agent, and returns the reward (utility) associated with selecting this action, i.e., $r\sim\mathcal{D}_{a}$. We'll use [a Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) and the Cobb-Douglas utility to model the rewards.

In [2]:
function world(a::Vector{Int64}, n::Dict{Int,Array{Float64,1}}, context::MyBanditConsumerContextModel)::Float64

    # initialize -
    γ = context.γ; # consumer preferences (unknown to bandits)
    σ = context.σ; # noise in utility calculation (unknown to bandits)
    B = context.B; # max budget (unknown to bandits)
    C = context.C; # unit costs of goods (unknown to bandits)
    λ = context.λ; # sensitivity to the budget
    Z = context.Z; # noise model
    ϵ = 0.001; # min unit required
    number_of_categories = context.m; # number of categories

    # compute the reward for this choice -
    Ū = 1.0;
    BC = 0.0;
    for i ∈ 1:number_of_categories
        
        # what action in category i, did we just take?
        aᵢ = a[i]; # this is which good to purchase in category i -
        nᵢ = max(ϵ, n[i][aᵢ]); # this is how much of good i to purchase (must be geq ϵ)
        Cᵢ = C[i][aᵢ]; # cost of chosen good in category i
        γᵢ = γ[i][aᵢ]; # preference of good in category i
        σᵢ = (σ[i][aᵢ]); # standard dev for good i
        Zᵢ = Z[i]; # noise model
   
        # update the utility -
        Ū *= nᵢ^(γᵢ)

        # compute the budget constraint -
        BC += nᵢ*Cᵢ;

        # @show i, aᵢ, nᵢ, Cᵢ, BC, Ū;

    end

    # compute the budget constraint violation -
    U = Ū - λ*max(0.0, (BC-B))^2; # use a penalty method to capture budget constraint

    # return the reward -
    return U;
end;

### Constants
Finally, let's set some constants we'll use in the subsequent tasks. See the comment beside the value for a description of what it is, its permissible values, etc.

In [3]:
T = 10000; # number of rounds for each decision task

## Task 2: Something will go here.
Fill me in

In [4]:
context = let

    # initialize -
    γ = Dict{Int,Vector{Float64}}(); # consumer preferences (unknown to bandits)
    σ = Dict{Int,Vector{Float64}}(); # noise in utility calculation (unknown to bandits)
    B = 100.0; # max budget (unknown to bandits)
    C = Dict{Int,Vector{Float64}}(); # unit costs of goods (unknown to bandits)
    λ = 0.0; # sensitivity to the budget
    Z = Dict{Int,Normal}(); # noise model
    number_of_categories = 3; # number of categories

    # set the parameters -
    # preferences
    γ[1] = [1.0, 1.0, 1.0]; # category 1
    γ[2] = [1.0, -1.0, 1.0, 1.0, 1.0, 1.0]; # category 2
    γ[3] = [1.0, 1.0, 1.0, 1.0]; # category 3

    # uncertainty
    σ[1] = [0.01, 0.01, 0.01]; # category 1
    σ[2] = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]; # category 2
    σ[3] = [0.01, 0.01, 0.01, 0.01]; # category 3

    # costs
    C[1] = [10.0, 20.0, 30.0]; # category 1
    C[2] = [10.0, 20.0, 30.0, 40.0, 50.0, 60.0]; # category 2
    C[3] = [10.0, 20.0, 30.0, 40.0]; # category 3

    # noise model
    Z[1] = Normal(0.0, 1.0); # category 1
    Z[2] = Normal(0.0, 1.0); # category 2
    Z[3] = Normal(0.0, 1.0); # category 3

    # build a context model with the reqired parameters -
    context = build(MyBanditConsumerContextModel, (
        γ = γ, # consumer preferences (unknown to bandits)
        σ = σ, # noise in utility calculation (unknown to bandits)
        B = B, # max budget (unknown to bandits)
        C = C, # unit costs of goods (unknown to bandits)
        λ = λ, # sensitivity to the budget
        Z = Z, # noise model
        m = number_of_categories
    )); # build the context

    # return 
    context;
end;

## Task 2: Evaluation of Algorithms
In this task, we'll run the $\epsilon$-greedy algorithm on our example `K`-arm bandit with categories problem. Let's start with a quick review of the $\epsilon$-greedy algorithm.

### Epsilon-Greedy Exploration
One issue with the _uniform exploration_ algorithm is that it may not be the best choice for all problems. For example, the performance in the exploration phase may be _bad_ if many of the arms have a large gap $\Delta({a})$:
* _What is this gap_? Let the (true) mean reward for each arm be $\mu(a) = \mathbb{E}\left[r_{t}\sim\mathcal{D}_{a}\right]$, where $a\in\mathcal{A}$. The _best_ mean reward over the actions is $\mu^{\star} = \max_{a\in\mathcal{A}}\mu_{a}$. Then, the gap $\Delta({a}) = \mu^{\star} - \mu(a)$ is the difference between the mean reward of the best arm and the mean reward of arm $a$. If the gap is _large_, the agent may miss out on many rewards by exploring each arm equally.

In a large gap, it may be better to spread out (and interleave) the exploration and exploitation phases of the arms. This is the idea behind the _epsilon-greedy_ algorithm. In this algorithm, the agent chooses the best arm with probability $1-\epsilon$ and a random arm with probability $\epsilon$. This allows the agent to explore the arms more evenly and may lead to better performance in cases where the gap is large.

While [Slivkins](https://arxiv.org/abs/1904.07272) doesn't give a reference for the epsilon-greedy algorithm, other sources point to (at least in part) to [Thompson and Thompson sampling, proposed in 1933 in the context of drug trials](https://arxiv.org/abs/1707.02038).

#### Epsilon-Greedy Algorithm
The agent has $K$ arms (choices), $\mathcal{A} = \left\{1,2,\dots,K\right\}$, and the total number of rounds is $T$. The agent uses the following algorithm to choose which arm to pull (which action to take) during each round:

For $t = 1,2,\dots,T$:
1. _Initialize_: Roll a random number $p\in\left[0,1\right]$ and compute a threshold $\epsilon_{t}\sim{t}^{-1/3}$. Note, in other sources, $\epsilon$ is a constant, not a function of $t$.
2. _Exploration_: If $p\leq\epsilon_{t}$, choose a random (uniform) arm $a_{t}\in\mathcal{A}$. Execute the action $a_{t}$ and receive a reward $r_{t}$ from the _adversary_ (nature). 
3. _Exploitation_: Else if $p>\epsilon_{t}$, choose action $a^{\star}$ (action with the highest average reward so far, the greedy choice). Execute the action $a^{\star}_{t}$ and recieve a reward $r_{t}$ from the _adversary_ (nature).
4. Update list of rewards for $a_{t}\in\mathcal{A}$

__Theorem__: The epsilon-greedy algowithm with exploration probability $\epsilon_{t}={t^{-1/3}}\cdot\left(K\cdot\log(t)\right)^{1/3}$ achives a regret bound of $\mathbb{E}\left[R(t)\right]\leq{t}^{2/3}\cdot\left(K\cdot\log(t)\right)^{1/3}$ for each round $t$.
___

We've modified the base algorithm to work category-wise. 

In [5]:
results, model = let

    # initialize -
    K = Dict{Int64,Int64}(); # arms dictionary
    n = Dict{Int64, Array{Float64,1}}() # items dictionary

    # How many alternatives (arms) do we have in category?
    K[1] = 3; # category 1 has three possible choices
    K[2] = 6; # categorty 2 has six possible choices
    K[3] = 4; # category 4 has four possible choices

    # how many items would we purchase *if* we choose alternative i in category j?  
    n[1] = [1.0, 1.0, 1.0]; # category 1
    n[2] = [2.0, 2.0, 2.0, 2.0, 2.0, 2.0]; # category 2
    n[3] = [3.0, 3.0, 3.0, 3.0]; # category 3
    
    # build model -
    m = build(MyEpsilonGreedyAlgorithmModel, (
        K = K, # arms dictionary
        n = n, # items dictionary
    ));
    
    # run the scenario, let's see what happens
    results = solve(m, T = T, world = world, context=context);
    
    # return -
    results, m;
end;

__What does the modified algorithm produce__? Let's build a table to see that choices that the bandits in each category are making. `Unhide` the code-block below to see how we construct the simulation results table.

In [6]:
let
    
    # initialize -
    df = DataFrame();
    number_of_categories = context.m; # number of categories
    category_action_map = model.K # get the number of arms
    B = context.B; # max budget (unknown to bandits)
    

    # loop over the categories -
    BC = 0.0;
    U = 1.0;
    for i ∈ 1:number_of_categories
    
        # Data for this categorty
        K = category_action_map[i]; # get the number of arms for this category
        data = results[i]; # get the data for this category

        μ = Array{Float64,1}(undef, K); # mean of the data
        for j ∈ 1:K
            μ[j] = filter(x -> x != 0.0, data[:,j]) |> x-> mean(x)
        end
       
        # which action should we take?
        aᵢ = argmax(μ); # this is which good to purchase in category i -
        BC += model.n[i][aᵢ]*context.C[i][aᵢ]; # budget constraint
        U *= model.n[i][aᵢ]^(context.γ[i][aᵢ]); # update the utility

        row_df = (
            category = i,
            action = aᵢ,
            n = model.n[i][aᵢ], # this is how much of good i to purchase (must be geq ϵ)
            unitcost = context.C[i][aᵢ], # cost of chosen good in category i
            spend = BC, # this is how much we spent
            remaining = B - BC, # budget constraint
            Ū = U, # utility
        );
        
        # add the row to the dataframe -
        push!(df, row_df);
    end

    pretty_table(df, tf = tf_simple);
end

=========== ======== ========= ========== ========= =========== ==========
  category   action         n   unitcost     spend   remaining         Ū 
     Int64    Int64   Float64    Float64   Float64     Float64   Float64 
=========== ======== ========= ========== ========= =========== ==========
         1        2       1.0       20.0      20.0        80.0       1.0
         2        1       2.0       10.0      40.0        60.0       2.0
         3        4       3.0       40.0     160.0       -60.0       6.0
=========== ======== ========= ========== ========= =========== ==========
